In [31]:
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

lakefs_endpoint = "http://localhost:8001/"
storage_options = {
    "key": os.getenv("ACCESS_KEY"),
    "secret": os.getenv("SECRET_KEY"),
    "client_kwargs": {
        "endpoint_url": lakefs_endpoint
    }
}
repo_name = "tweets-repo"
branch_name = "main"
path = "tweets.parquet"
lakefs_s3_path = f"s3://{repo_name}/{branch_name}/{path}"
df = pd.read_parquet(
    lakefs_s3_path,
    storage_options=storage_options,
    engine='pyarrow',
)
df.sort_values(by=['postTimeRaw'], ascending=True, inplace=True)
df.drop_duplicates(subset="tweetText")
df['index'] = df.index + 1
df['postTimeRaw'] = df['postTimeRaw'].dt.strftime('%Y-%m-%d')
print(df.shape)
display(df.tail(5))
display(df.head(5))


FileNotFoundError: tweets-repo/main/tweets.parquet/year=2019/month=1/day=30/1585f9c156584d668f3fb6cbfbc5653e-0.parquet

In [9]:
df_dict:dict = df[['postTimeRaw', 'tweetText', 'index']].to_dict(orient='records')
print(df_dict[0:5])

[{'postTimeRaw': '2018-10-01', 'tweetText': "Scholars at TU's Institute for Trauma, Adversity and Injustice including law professor Mimi Marton, offer clarity about the nature of interpersonal violence to combat misinformation. #TULaw #utulsa http://bit.ly/2Nfp9mP", 'index': 1}, {'postTimeRaw': '2018-10-02', 'tweetText': 'Would you like to know more about attending the #1 best value private law school in the U.S.? The admissions team at TU Law is holding an informational session on Wednesday, Oct. 3, from noon-1 p.m. at 3120 E. 4th Place in Tulsa, Room 2442. #utulsa #TULaw http://bit.ly/2NPt6Uo', 'index': 8}, {'postTimeRaw': '2018-10-03', 'tweetText': 'The latest issue of the Energy Law Journal is available. Through a partnership between TU Law students & the Energy Bar Association, this peer-reviewed legal publication provides relevant commentary on regulatory and energy issues. #TULaw #utulsa http://bit.ly/2OB6cjy', 'index': 13}, {'postTimeRaw': '2018-10-03', 'tweetText': 'The latest

In [12]:
import os 
from dotenv import load_dotenv

load_dotenv()

GEMINI_API_KEY  = os.getenv("GEMINI_API_KEY")
os.environ["GEMINI_API_KEY "] = GEMINI_API_KEY 

from google import genai
from google.genai import types

import os
from dotenv import load_dotenv
from google import genai
from google.genai import types

# Load API key
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

# Initialize client
client = genai.Client(api_key=GEMINI_API_KEY)

instruction = """
คุณทำหน้าที่ในฝ่ายประชาสัมพันธ์ของมหาวิทยาลัย เป้าหมายของคุณคือการรวบรวมและจัดกลุ่ม 
"คำถามที่พบบ่อย" (FAQ) หรือ "ปัญหาที่พบบ่อย" (Issue) จากโซเชียลมีเดีย 
เพื่อใช้ในการตัดสินใจว่าควรสื่อสารผ่าน PR หรือรายงานต่อหน่วยงานที่เกี่ยวข้อง
โดยปัญหา / คำถามที่นำมาจัดกลุ่มจะต้องเกี่ยวข้องและแก้ไขได้ในระดับมหาวิทยาลัย
คุณจะได้รับข้อความจากโซเชียลมีเดียที่เกี่ยวข้องกับมหาวิทยาลัย

คำแนะนำในการจัดกลุ่ม:
1. **ระบุประเภท**: แยกระหว่าง "คำถาม" (faq) และ "ปัญหา" (issue)
   - คำถาม (faq): ข้อความที่ผู้ใช้ต้องการข้อมูลหรือคำแนะนำ / ประโยคคำถาม **ต้องเป็นคำถามที่ทางมหาวิทยาลัยสามารถตอบได้** และ เกี่ยวข้องกับทางมหาวิทยาลัยโดยตรง
   - ปัญหา (issue): ข้อความที่ผู้ใช้ระบุถึงความไม่พึงพอใจหรือสิ่งที่ต้องแก้ไข **ต้องเป็นปัญหาที่ทางมหาวิทยาลัยสามารถแก้ไขได้** และ เกี่ยวข้องกับทางมหาวิทยาลัยโดยตรง

2. **ระบุหมวดหมู่ (topic)**:
   - ใช้หมวดหมู่ที่มีอยู่แล้วหากข้อความใหม่เข้ากับหมวดหมู่เดิม
   - สร้างหมวดหมู่ใหม่เมื่อไม่มีหมวดหมู่เดิมที่เหมาะสม
   - ตั้งชื่อหมวดหมู่ให้กระชับ เข้าใจง่าย และมีความเฉพาะเจาะจงในระดับที่เหมาะสม
   - ข้อความหนึ่งสามารถอยู่ได้หลายหมวดหมู่หากมีความเกี่ยวข้อง
   - ต้องเกี่ยวข้องกับทางมหาวิทยาลัยโดยตรง และสามารถแก้ไขได้ในระดับมหาวิทยาลัย

3. **ระบุหมวดหมู่ย่อย (subtopic)**:
   - ระบุหมวดหมู่ย่อยที่มีความเฉพาะเจาะจงมากขึ้น
   - สามารถมีได้หลายหมวดหมู่ย่อยต่อหนึ่งข้อความ
   - หมวดหมู่ย่อยควรให้รายละเอียดเพิ่มเติมที่เป็นประโยชน์เกี่ยวกับคำถามหรือปัญหานั้น ๆ
   - ต้องเกี่ยวข้องกับทางมหาวิทยาลัยโดยตรง และสามารถแก้ไขได้ในระดับมหาวิทยาลัย

4. **พิจารณาเฉพาะข้อความที่เกี่ยวข้อง**:
   - พิจารณาเฉพาะข้อความที่เป็นคำถามหรือปัญหาเท่านั้น
   - ข้อความทั่วไป ข้อความสนทนา หรือข้อความที่ไม่มีเนื้อหาเป็นคำถามหรือปัญหา ไม่ต้องนำมาจัดกลุ่ม

ตอบกลับมาในรูปแบบ JSON โดยมีโครงสร้างดังนี้:
{
    "issue": [
        {"index": 1, "text": "ข้อความ", "topic": ["หมวดหมู่1", "หมวดหมู่2"], "subtopic": ["หมวดย่อย1", "หมวดย่อย2"]},
        {"index": 2, "text": "ข้อความ", "topic": ["หมวดหมู่1"], "subtopic": ["หมวดย่อย1"]}
    ],
    "faq": [
        {"index": 1, "text": "ข้อความ", "topic": ["หมวดหมู่1"], "subtopic": ["หมวดย่อย1", "หมวดย่อย2"]},
        {"index": 2, "text": "ข้อความ", "topic": ["หมวดหมู่1", "หมวดหมู่2"], "subtopic": ["หมวดย่อย1"]}
    ]
}

หากไม่มีคำถามหรือปัญหาที่เกี่ยวข้อง ให้ส่งคืนค่าเป็น empty array ในหมวดนั้น
"""

# Template for use with input data
prompt_template = """
# topic ของคำถามที่พบบ่อยในอดีต (FAQ) - ใช้เป็นตัวอย่างในการจัดกลุ่ม:
{faq_topic}

# sub topic ของคำถามที่พบบ่อยในอดีต (FAQ) - ใช้เป็นตัวอย่างในการจัดกลุ่ม:
{faq_subtopic}

# topic ของปัญหาที่พบบ่อยในอดีต (Issue) - ใช้เป็นตัวอย่างในการจัดกลุ่ม:
{issue_topic}

# sub topic ของปัญหาที่พบบ่อยในอดีต (Issue) - ใช้เป็นตัวอย่างในการจัดกลุ่ม:
{issue_subtopic}

# ตัวอย่างการจัดกลุ่ม:
text: 'ไฟล์สมัครในเว็บมธ.อยู่ตรงไหนเหรอคะ มีใครพอจะทราบไหมคะ' 
topic: ['สอบถามเอกสาร']
subtopic: ['เอกสารการสมัคร', 'การเข้าถึงข้อมูล']

text: 'หอในเปิดปิดกี่โมง มีเคอร์ฟิวไหม แล้วถ้าเข้าหอดึกต้องทำยังไงบ้าง'
topic: ['หอพัก']
subtopic: ['กฎระเบียบหอพัก', 'เวลาเปิด-ปิด']

text: 'ระบบลงทะเบียนล่มอีกแล้ว ทำไมเกิดปัญหาทุกเทอมเลย'
topic: ['ระบบลงทะเบียน', 'ปัญหาเทคนิค']
subtopic: ['ระบบล่ม', 'ความเสถียรของระบบ']

# ข้อความที่ต้องการจัดกลุ่ม:
{messages}

โปรดวิเคราะห์และจัดกลุ่มข้อความตามคำแนะนำที่ให้ไว้ และส่งคืนเป็น JSON ตามรูปแบบที่กำหนด
"""

In [13]:
import json

faq_topic, faq_subtopic = set(), set()
issue_topic, issue_subtopic = set(), set()

def classify_messages(
    tweets_eles: list,
    faq_topic: str = faq_topic,
    faq_subtopic: str = faq_subtopic,
    issue_topic: str = issue_topic,
    issue_subtopic: str = issue_subtopic
    ) -> dict:
    """
    Classify messages into issues and FAQs.
    
    Args:
        messages (list): List of messages to classify.
        
    Returns:
        dict: Classified messages in JSON format.
    """
    prompt_formatted = prompt_template.format(
        faq_topic = faq_topic,
        faq_subtopic = faq_subtopic,
        issue_topic = issue_topic,
        issue_subtopic = issue_subtopic,
        messages="\n".join([f"{row['index']}: {row['tweetText']}" for row in tweets_eles]),
    )
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt_formatted,
        config=types.GenerateContentConfig(
            system_instruction=instruction,
            temperature=0.2, # low temperature for more deterministic output kub
        ),
    )
    response_text = response.text
    response_json = response_text[response_text.index("{"): response_text.rindex("}") + 1]
    response_json = response_json.replace("{{", "{").replace("}}", "}")
    response_json = json.loads(response_json, strict=False)
    return response_json

In [15]:
first_50_rows = df_dict[0:50]
first_50_rows_response = classify_messages(first_50_rows)

print("FAQ :")
print(first_50_rows_response['faq'][0:5])
print("="* 50)
print("Issue :")
print(first_50_rows_response['issue'][0:5])
print("="* 50)

FAQ :
[{'index': 26, 'text': 'ตามหา #เสื้อเชียร์ธรรมศาสตร์ ไซส์ XL 1ตัวค่าาาา ต้องการอย่างมากๆๆๆๆๆๆๆ มีใครรับหิ้ว หรือจะขายต่อ เชิญติดต่อทางนี้ด่วนค่าาาาา\nID LINE : 0941478342 \nหรือdm มาก้ได้ ตอบเร็วมากกก\n#เสื้อเชียร์ธรรมศาสตร์ \n#tu83 #tucheer \n#tu #ทีมธรรมศาสตร์\n#มธ #นิติมธ #tulaw', 'topic': ['สินค้าและบริการ'], 'subtopic': ['เสื้อเชียร์', 'ตามหา/ซื้อขาย']}, {'index': 46, 'text': 'ข้อสอบเรียงความสอบตรงนิติมธ.61\nการอนุญาตใช้กัญชาในประเทศไทย\nติวสอบตรงนิติมธ.&LLB 2563 ม.4,5,6 เริ่มได้แล้วน๊า \nสอบถาม  https://line.me/R/ti/p/%40geniuslawtu…\n#สอบตรงนิติมธ #นิติศาสตร์ #นิติมธ #ทีมมธ #ทีมนิติมธ #geniuslawtu #tcas62 #tcas63 #tcas64 #dek63 #gatpat #tulaw #tu', 'topic': ['การสอบ', 'หลักสูตร'], 'subtopic': ['สอบตรง', 'นิติศาสตร์', 'LLB']}, {'index': 47, 'text': 'ข้อสอบจริงคณะนิติมธ.หลักประชาธิปไตย\nติวสอบตรงนิติมธ & LLB 2563\nม.4,5,6 ห้ามพลาด\nLINE https://line.me/R/ti/p/%40tsb8010k… \n#สอบตรงนิติมธ #นิติมธ #lawtu #ทีมมธ #ทีมนิติมธ #gatpat #onet #dek63 #tcas62 #tcas63 #tcas64 #tcas65 #d

In [22]:
step = 50
prev_stop = 0

all_response = []

for ind in range(step, len(df_dict) + step, step):
    start = prev_stop
    stop = ind
    prev_stop = stop
    rows = df_dict[start:stop]
    print(f"Processing rows {start} to {stop}")
    
    response = classify_messages(rows)  # already a dict
    all_response.append(response)

    # Update the sets
    for row in response.get('issue', []):
        for topic in row.get('topic', []):
            issue_topic.add(topic)
        for subtopic in row.get('subtopic', []):
            issue_subtopic.add(subtopic)
    
    for row in response.get('faq', []):
        for topic in row.get('topic', []):
            faq_topic.add(topic)
        for subtopic in row.get('subtopic', []):
            faq_subtopic.add(subtopic)


Processing rows 0 to 50
Processing rows 50 to 100
Processing rows 100 to 150
Processing rows 150 to 200
Processing rows 200 to 250
Processing rows 250 to 300
Processing rows 300 to 350
Processing rows 350 to 400
Processing rows 400 to 450
Processing rows 450 to 500
Processing rows 500 to 550
Processing rows 550 to 600
Processing rows 600 to 650
Processing rows 650 to 700
Processing rows 700 to 750
Processing rows 750 to 800
Processing rows 800 to 850
Processing rows 850 to 900
Processing rows 900 to 950
Processing rows 950 to 1000
Processing rows 1000 to 1050


In [28]:

today = pd.Timestamp.now().strftime("%Y-%m-%d")

faqs = [faq for response in all_response for faq in response['faq']  ]
faqs_df = pd.DataFrame(faqs)
faqs_df = faqs_df.merge(
    df[['index', 'postTimeRaw']],
    how='left',
    on='index'
)
faqs_df.to_csv(f"{today}.csv", index=False)

issues = [issue for response in all_response for issue in response['issue']]
issues_df = pd.DataFrame(issues)
issues_df = issues_df.merge(
    df[['index', 'postTimeRaw']],
    how='left',
    on='index'
)
issues_df.to_csv(f"{today}.csv", index=False)

In [29]:
display(faqs_df.head())
display(issues_df.head())

,index,text,topic,subtopic,postTimeRaw
0,26,ตามหา #เสื้อเชียร์ธรรมศาสตร์ ไซส์ XL 1ตัวค่าาา...,[สินค้าและบริการ],"[เสื้อเชียร์, ตามหา/ซื้อขาย]",2019-01-10
1,42,ขอเชิญนักศึกษามหาวิทยาลัยธรรมศาสตร์ทุกคณะทุกชั...,"[กิจกรรม, นิติศาสตร์]","[กิจกรรม, วิ่ง]",2019-02-06
2,44,อังคารหน้าห้ามพลาด! เสวนาเปิดมุมมองและความท้าท...,"[กิจกรรม, นิติศาสตร์]","[เสวนา, กฎหมายระหว่างประเทศ]",2019-03-15
3,46,ข้อสอบเรียงความสอบตรงนิติมธ.61\nการอนุญาตใช้กั...,"[การสอบ, นิติศาสตร์]","[แนวข้อสอบ, ติวสอบ, สอบตรงนิติมธ, tcas62, tcas...",2019-03-28
4,47,ข้อสอบจริงคณะนิติมธ.หลักประชาธิปไตย\nติวสอบตรง...,"[การสอบ, นิติศาสตร์]","[แนวข้อสอบ, ติวสอบ, สอบตรงนิติมธ, tcas62, tcas...",2019-03-29


,index,text,topic,subtopic,postTimeRaw
0,116,#tulaw #ม็อบ17ตุลา #whatishappeninginthailand,"[การเมือง, สังคม]","[การเมือง, สังคม]",2020-10-17
1,182,อยากสอบถามค่ะะะ พอดีเราอยากแก้ไขไฟล์ที่ส่งไปให...,"[ระบบรับสมัคร, TCAS]","[ระบบสมัคร, การแก้ไขข้อมูล]",2023-01-16
2,221,#ธรรมศาสตร์ช้างเผือก มันไม่ขึ้นให้ตรวจสอบอ่ะค่...,"[ระบบรับสมัคร, TCAS, การเข้าถึงข้อมูล]","[ระบบสมัคร, การเข้าถึงข้อมูล, ระบบล่ม]",2023-02-14
3,189,ธรรมศาสตร์ช้างเผือกถ้าหาเกรดประถมกับม.ต้นไม่เจ...,"[TCAS, การรับสมัคร]","[เอกสาร, เอกสารการสมัคร]",2023-01-09
4,212,ทำไมเว็บล่มอะ กลัวเขาแก้ไขรายชื่อละรี่หลุด #ธ...,"[ระบบรับสมัคร, TCAS]","[ระบบล่ม, ความเสถียรของระบบ]",2023-02-14
